In [1]:
# TO INSTALL pytorch3D TO COLAB
import os
import sys
import torch
need_pytorch3d=False
try:
    import pytorch3d
except ModuleNotFoundError:
    need_pytorch3d=True
if need_pytorch3d:
    if torch.__version__.startswith("1.11.") and sys.platform.startswith("linux"):
        # We try to install PyTorch3D via a released wheel.
        pyt_version_str=torch.__version__.split("+")[0].replace(".", "")
        version_str="".join([
            f"py3{sys.version_info.minor}_cu",
            torch.version.cuda.replace(".",""),
            f"_pyt{pyt_version_str}"
        ])
        !pip install fvcore iopath
        !pip install --no-index --no-cache-dir pytorch3d -f https://dl.fbaipublicfiles.com/pytorch3d/packaging/wheels/{version_str}/download.html
    else:
        # We try to install PyTorch3D from source.
        !curl -LO https://github.com/NVIDIA/cub/archive/1.10.0.tar.gz
        !tar xzf 1.10.0.tar.gz
        os.environ["CUB_HOME"] = os.getcwd() + "/cub-1.10.0"
        !pip install 'git+https://github.com/facebookresearch/pytorch3d.git@stable'

### CMPE538. Project 3.A
We use two popular 3D representations in this project: Mesh and Point Clouds. We create 3D objects from scratch and understand the basics of 3D data structures. 
We plot these objects and apply some basic operations to them. 

Tutorials:
- https://pytorch3d.readthedocs.io/en/latest/modules/index.html


Please fill your info.

Name/Surname: Emre Girgin


E-mail: emre.girgin@boun.edu.tr

Student No: 2021700060

In [2]:
import argparse
import os
import time
import torch
import numpy as np


from pytorch3d.utils import ico_sphere
from pytorch3d.ops import sample_points_from_meshes
from pytorch3d.renderer import TexturesVertex # Use to add texture to Mesh object

from pytorch3d.structures import Meshes, Pointclouds 
from pytorch3d.vis.plotly_vis import AxisArgs, plot_batch_individually, plot_scene

torch.manual_seed(42)
np.random.seed(42)

# Build Mesh from scratch

In this example, we visualize 3D body/face/hands in 3D space


In [3]:
# Define MESH manually: Vertices, Faces
# Define Vertices
# Vertices will be the corner of a Cube : Cube edge length is 2. One corner will be at (0,0,0),
# locate remaining corners  +x,+y,+z region of R3.
###################################################################
# HERE !
verts = np.array([
                  [0,0,0],
                  [0,0,2],
                  [0,2,0],
                  [0,2,2],
                  [2,0,0],
                  [2,0,2],
                  [2,2,0],
                  [2,2,2]
                ]).astype(np.float)
###################################################################


# Define Faces. Face holds the vertice indexes. Order of vertices in face is very very important. Use right hand rule.
# While defining the order of vertices in face, be sure about that normal vector of the face points out the cube.(your thumb points our cube.)
# [0,1,2] !=[2,1,0] Two face have different Normal vectors.

# Divide each face of Cube into 2 triangles                  
# check image: https://miro.medium.com/max/570/1*mLeAh-bGdY5N_uN96ujjWw.png   
# Don't consider the vertex indexing in the image.
###################################################################
# HERE !
faces = np.array([
                  [1,7,3],
                  [1,5,7],
                  [5,6,7],
                  [5,4,6],
                  [3,7,2],
                  [7,6,2],
                  [4,2,6],
                  [4,0,2],
                  [0,1,3],
                  [0,3,2],
                  [0,4,5],
                  [0,5,1]
                   ]).astype(np.float32)
###################################################################

# Numpy -> Torch
verts_tensor = torch.from_numpy(verts)
faces_tensor = torch.from_numpy(faces)

# Create Pytorch3d Mesh object [vertices,faces, but without texture ]
my_mesh = Meshes(verts=[verts_tensor], 
                 faces=[faces_tensor])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  app.launch_new_instance()


### Plot Mesh

In [4]:
# Ploty: "my_mesh" without background
fig = plot_scene({
    "subplot1": {
        "PC_SRC": my_mesh
    }
})
fig.show()

### Plot Mesh with Background

In [5]:
# Ploty: "my_mesh" with background
fig = plot_scene({
    "subplot1": {
        "PC_SRC": my_mesh
}},
    xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
    yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
    zaxis={"backgroundcolor":"rgb(200, 230, 200)"}, 
    axis_args=AxisArgs(showgrid=True))

fig.show()

### Adding texture to Mesh Object

In [6]:
def addTexture2Mesh(_mesh):
    __mesh = _mesh.clone()
    
    verts_rgb = torch.rand((1,__mesh._V,3))  # (1, V, 3)

    textures = TexturesVertex(verts_features=verts_rgb)
    
    __mesh.textures = textures
    
    return __mesh


In [7]:
# Add random color texture to Mesh Object.
mesh_textured = addTexture2Mesh(my_mesh)
mesh_non_textured = my_mesh

########################################################
# Render Mesh without texture and Mesh with 'random colored texture'
fig = plot_scene({
    "subplot1_textured": {
        "mesh_textured": mesh_textured
    },
       "subplot2_no_texture": {
        "mesh_non_textured": mesh_non_textured
    }},
)

fig.show()

# Find Mesh Centroid (face-area weighted)

Calculate the centroid of "my_mesh" object. Consider the area sizes of each face. First, find the centroid of each face, then apply the weighted sum to face-centroids to get the object-centroid.

In [43]:
def mesh_weighted_centroid(my_mesh):
    verts, faces = my_mesh.get_mesh_verts_faces(0)
    ###################################################################
    # HERE !
    total_area = 0
    centers = torch.zeros(3)
    for face in faces:
        triangle_normal = np.cross(
            verts[face[1]] - verts[face[0]],
            verts[face[2]] - verts[face[0]]
        ) 
        area = np.linalg.norm(triangle_normal) / 2 
        centers+= torch.mean(verts[face], axis=0) * area
        total_area += area
 
    centers /= total_area
    centroid = centers.tolist()
    ###################################################################
    return centroid 

In [44]:
my_centroid = mesh_weighted_centroid(my_mesh)
print(f'centroid: {my_centroid}')

centroid: [1.0, 1.0, 1.0]


# Calculate Face Normals & Vertex normals

Face Normals: Calculate the normal for all faces by taking the cross product of the vectors v1-v0 and v2-v0.
Vertex Normals: Calculate the normal for all vertex by taking the average of faces normals that a vertex contributes. You will need face normals to find vertex normals. So, begin with "face normals". While computing vertex normals, do not consider the areas of triangles.(Unweigthed averaging)

Be aware: Normal vectors should have unit length. Be sure about normalization.


### Face normals (Unweighted)

In [10]:
#This a function to normalize normal vectors
def normalizeV(arr):
    #Normalize a numpy vector (3x1) 
    ###################################################################
    # HERE !

    arr /= np.linalg.norm(arr)
  
    ###################################################################              
    return arr

In [11]:
# mesh_obj: Pytorch3d Mesh Obj. Returns: Fx3 (face normal vectors) (F: num of triangles/faces)
def calcFaceNormals(mesh_obj):
    verts, faces = mesh_obj.get_mesh_verts_faces(0)
    face_num = mesh_obj._F
    face_norm = np.zeros( (face_num,3), dtype=np.float32 )

    ###################################################################
    # HERE !

    for idx, face in enumerate(faces):
        vertices = verts[face]

        face_norm[idx] = normalizeV(np.cross(
            vertices[1] - vertices[0],
            vertices[2] - vertices[0]
        ))
  
    ###################################################################
    
    return face_norm

In [12]:
calcFaceNormals(my_mesh)

array([[ 0.,  0.,  1.],
       [ 0.,  0.,  1.],
       [ 1., -0.,  0.],
       [ 1.,  0.,  0.],
       [-0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0., -1.],
       [ 0.,  0., -1.],
       [-1.,  0.,  0.],
       [-1.,  0.,  0.],
       [ 0., -1.,  0.],
       [ 0., -1.,  0.]], dtype=float32)

### Vertex normal (face - unweighted)

In [13]:
# mesh_obj: Pytorch3d Mesh Obj. Returns: Vx3 (normal vectors) (V: num of vertex)
def calcVertexNormals(mesh_obj):
    verts, faces = mesh_obj.get_mesh_verts_faces(0)
    verts_num = mesh_obj._V
    verts_normal = np.zeros( (verts_num,3), dtype=np.float32 )
    ###################################################################
    # HERE !

    face_normals = calcFaceNormals(my_mesh)

    for vert_id, vert in enumerate(verts):
        neighbor_normals = []
        for face_id, face in enumerate(faces):
            if vert_id in face:
                neighbor_normals.append(face_normals[face_id])

        verts_normal[vert_id] = np.asarray(neighbor_normals).mean(axis=0)
    
  
    ###################################################################
    return verts_normal

In [14]:
calcVertexNormals(my_mesh)

array([[-0.4 , -0.4 , -0.2 ],
       [-0.25, -0.25,  0.5 ],
       [-0.2 ,  0.4 , -0.4 ],
       [-0.5 ,  0.25,  0.25],
       [ 0.25, -0.25, -0.5 ],
       [ 0.4 , -0.4 ,  0.2 ],
       [ 0.5 ,  0.25, -0.25],
       [ 0.2 ,  0.4 ,  0.4 ]], dtype=float32)

# Build Point Cloud

In this example, we visualize 3D body/face/hands in 3D space

In [15]:
#Function: Takes "numpy point coord array Vx3", Returns 'PointClouds' obj. (pytorch3d)
def numpy2PC(point_arrray):
    
    ###################################################################
    # HERE !
    PC_ = Pointclouds(torch.from_numpy(point_arrray))
    ###################################################################
    
    return PC_ # PC_: 'PointClouds' obj. (pytorch3d)
    

In [16]:
# Define Point Clouds(PC) manually
# Points will be the corner of a Cube : Cube edge length is 2. One corner will be at (0,0,0),
# locate remaining corners  +x,+y,+z region of R3.

###################################################################
# HERE !
PC_1_coords = np.array([[
                  [0,0,0],
                  [0,0,2],
                  [0,2,0],
                  [0,2,2],
                  [2,0,0],
                  [2,0,2],
                  [2,2,0],
                  [2,2,2]
                ]]).astype(np.float)
###################################################################


#
PC_1 = numpy2PC(PC_1_coords)



# Render the plotly figure
fig = plot_scene({
    "subplot1": {
        "PC_SRC": PC_1,
    }
})
fig.show()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



# Point Cloud and Permutation Invariance

In this example, we visualize 3D body/face/hands in 3D space

In [17]:
# Number of point in Point Cloud object
NumOfPoints = PC_1._P

# Permutate the order of point list.
my_rand_1 = np.random.permutation(NumOfPoints)
permuted1_PC1 = PC_1_coords[0,my_rand_1,:] # EDITED
permuted1_PC1_ = numpy2PC(np.expand_dims(permuted1_PC1, axis=0)) # EDITED


# Permutate the order of point list.
my_rand_2 = np.random.permutation(NumOfPoints)
permuted2_PC1 = PC_1_coords[0, my_rand_2,:] # EDITED
permuted2_PC1_ = numpy2PC(np.expand_dims(permuted2_PC1, axis=0)) # EDITED



# # Render PC_1 and permuted two versions together in one subplot. Are they the same object? Validate your understanding.
fig = plot_scene({
    "subplot1": {
        "PC1_SRC": PC_1,
        "PC1_permuted_1": permuted1_PC1_,
        "PC1_permuted_2": permuted2_PC1_,
    }
})

fig.show()


# Implement Chamfer Distance 
Restriction: Only numpy/torch tensor operations, implement video.

A visual representation of the Chamfer distance function: https://www.youtube.com/watch?v=P4IyrsWicfs (Visual, intuitive)

In [18]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Takes two point cloud objects. x: PC , y: PC . Returns : Scalar distance value. 
def chamfer_distance(x_PC, y_PC):
    x = x_PC.get_cloud(0)[0] # Pytorch3d PointCloud -> Vx3 tensor/array for point coords.
    y = y_PC.get_cloud(0)[0] # Pytorch3d PointCloud -> Vx3 tensor/array for point coords.
    ###################################################################
    # HERE !

    chamfer_dist = 0
    for p in x:
        chamfer_dist += np.min(np.linalg.norm(y-p, axis=1)**2)
    
    for q in y:
        chamfer_dist += np.min(np.linalg.norm(x-q, axis=1)**2)


    ###################################################################

    return chamfer_dist




In [19]:
# Find Chamfer Distance between PC_1 and one of Permutated versions.
cham_dist = chamfer_distance(PC_1,permuted1_PC1_)
print(cham_dist) # Expected value 0.0, if not check your implementation.

0.0


# Plot Point Cloud & Mesh together


###  Sampling From Mesh

Sample points on (pytorch3d) Mesh object.

In [20]:
from pytorch3d.ops import sample_points_from_meshes

my_mesh.verts_packed() # EDITED
my_mesh._verts_packed = my_mesh._verts_packed.float() # EDITED

# Sample 5000 Points on the surface of our 'my_mesh' Mesh object. Use "sample_points_from_meshes"
sampled_points = sample_points_from_meshes(my_mesh, num_samples=5000)
PC_from_my_mesh = Pointclouds(points=sampled_points)



fig = plot_scene({
    "subplot1": {
        "my_mesh": my_mesh,
        "Sampled Points": PC_from_my_mesh,
    }},
    xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
    yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
    zaxis={"backgroundcolor":"rgb(200, 230, 200)"}, 
    axis_args=AxisArgs(showgrid=True))


fig.show()

In [21]:
# Plot function. Takes - List(Mesh_obj1,Mesh_obj2,..), Then plot column by column.
def plot_mesh_list(mesh_list):
    
    verts_list, faces_list = [], []
    for i_mesh in mesh_list:
        verts, faces = i_mesh.get_mesh_verts_faces(0)
        verts_list.append(verts)  
        faces_list.append(faces)
        
    mesh_batch = Meshes(
        verts= verts_list,   
        faces=faces_list )
    
   
    figure = plot_batch_individually(
        mesh_batch, 
        ncols=len(mesh_list),
        #subplot_titles = None , # customize subplot titles
        xaxis={"backgroundcolor":"rgb(200, 200, 230)"},
        yaxis={"backgroundcolor":"rgb(230, 200, 200)"},
        zaxis={"backgroundcolor":"rgb(200, 230, 200)"}, 
        axis_args=AxisArgs(showgrid=True))
    
    figure.show()

In [22]:
# Try - Mesh List Plot - 
plot_trial = [my_mesh,my_mesh,my_mesh]
plot_mesh_list(plot_trial)

# Mesh Smoothing

In some cases, we need smoothing operations on mesh data. 

We first import a ready-to-use sphere mesh object from a built-in library: ico_sphere. Then, we add some noise (displacement) to each vertex of the sphere-mesh. 
If we plot sphere-mesh and noised-sphere together, we will notice the sparky surface of noised-version. We will apply a smoothing operation to make the noise-sphere smoother. Keep in mind that it will not be perfectly smooth again.


In [23]:
from pytorch3d.utils import ico_sphere

# import a Sphere mesh with Resolution-3 (we can change 1: lower resolution, 4-5 higher resolution)
sphere_mesh = ico_sphere(3)# in our project, keep resolution 3.


# Add Noise to Sphere Mesh
verts, faces = sphere_mesh.get_mesh_verts_faces(0)
noise = torch.randn((verts.shape))*0.2
noised_sphere = Meshes(verts=[verts + noise],  faces=[faces])

plot_mesh_list([sphere_mesh,noised_sphere])

/usr/local/lib/python3.7/dist-packages/pytorch3d/structures/meshes.py:1108: UserWarning:

__floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').



### Applying Laplacian Smoothing to vertices in a mesh

https://codereview.stackexchange.com/questions/159621/applying-laplacian-smoothing-to-vertices-in-a-mesh

Laplacian smoothing is an algorithm to smooth a polygonal mesh. For each vertex in a mesh, a new position is chosen based on local information (such as the position of neighbors) and the vertex is moved there.(src: wikipedia)

Hint: Find neighboring vertices in mesh. Sum their coordinates. Divide by the number of neighboring vertices. 


In [24]:
# Takes  as input. Smooth it. Returns a Pytorch3D Mesh (smoothed)
# Implement the function. Do Not call a similar function from a library. 
def MeshLaplacianSmoothing(mesh_obj):
    
    n_verts, n_faces = mesh_obj.get_mesh_verts_faces(0)
    
    ###################################################################
    # HERE !

    smoothed_verts = np.zeros_like(mesh_obj.verts_packed())

    for vert_id, vert in enumerate(mesh_obj.verts_packed()):
        neighbors = []
        for face_id, face in enumerate(mesh_obj.faces_packed()):
            if vert_id in face:
                neighbors += verts[face[face != vert_id]].tolist()

        smoothed_verts[vert_id] = np.asarray(neighbors).mean(axis=0)

    
    smoothed_verts_tensor = torch.from_numpy(smoothed_verts)
    smoothed_faces_tensor = mesh_obj.faces_packed()

    # Create Pytorch3d Mesh object [vertices,faces, but without texture ]
    smoothed_mesh = Meshes(verts=[smoothed_verts_tensor], 
                    faces=[smoothed_faces_tensor])
    ###################################################################
    
    return smoothed_mesh


In [25]:
#Apply Laplacian Smoothing to noised-sphere 
smoothed_mesh = MeshLaplacianSmoothing(noised_sphere)


#Apply Laplacian Smoothing to noised-sphere second times.
smoothed_mesh2 = MeshLaplacianSmoothing(smoothed_mesh)

# Plot : sphere_mesh, Noised Mesh, Laplacian Smoothed (1times), Laplacian Smoothed (2 times)
plot_mesh_list([sphere_mesh,noised_sphere,smoothed_mesh,smoothed_mesh2])

## Sample Points on the Meshes

In [26]:
sphere_points = sample_points_from_meshes(sphere_mesh,num_samples=5000)
sphere_PC = Pointclouds(points=sphere_points)

noised_mesh_points = sample_points_from_meshes(noised_sphere,num_samples=5000)
noised_sphere_PC = Pointclouds(points=noised_mesh_points)

smoothed_mesh_points = sample_points_from_meshes(smoothed_mesh,num_samples=5000)
smoothed_mesh_PC = Pointclouds(points=smoothed_mesh_points)

smoothed_mesh_points2 = sample_points_from_meshes(smoothed_mesh2,num_samples=5000)
smoothed_mesh_PC2 = Pointclouds(points=smoothed_mesh_points2)

### Calculate Chamfer distance btw:
#### Sphere_PC vs Sphere_PC (itself)  ~~ expected 0.0
#### a = Sphere_PC vs noised_mesh_PC
#### b = Sphere_PC vs smoothed_mesh_PC 
#### c = Sphere_PC vs smoothed_mesh_PC2 (2 times smoothed)
We expect a>b>c

In [27]:
chamfer_distance(sphere_PC,sphere_PC)

0.0

In [28]:
chamfer_distance(sphere_PC,noised_sphere_PC)

133.2183177857696

In [29]:
chamfer_distance(sphere_PC,smoothed_mesh_PC)

9.363947697755066

In [30]:
chamfer_distance(sphere_PC,smoothed_mesh_PC2)

9.017731288186042

### References
1 -

2 -

3 -
